In [2]:
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
#from matplotlib_venn import venn2, venn2_circles
from sklearn.cross_validation import cross_val_score
from bayes_opt import BayesianOptimization
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import scipy
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import Ridge
from sklearn.cross_validation import KFold
import time

NFOLDS = 5
SEED = 42
#-------------------------------------
test = pd.read_csv('/home/g492652607/data/test.csv')
train = pd.read_csv('/home/g492652607/data/train.csv')
gp = pd.read_csv('/home/g492652607/data/aggregated_features.csv') 
train = train.merge(gp, on='user_id', how='left')
test = test.merge(gp, on='user_id', how='left')

ntrain = train.shape[0]
ntest = test.shape[0]
agg_cols = list(gp.columns)[1:]

del gp
gc.collect()
#------------------------------------
blend = pd.read_csv('/home/g492652607/data/blend.csv') 
train=train.merge(blend, on='item_id', how='left')
test=test.merge(blend, on='item_id', how='left')
idx=blend['item_id']
del blend 
gc.collect()
#----------------------------------
for df in [train, test]:
    df['description'].fillna('unknowndesc', inplace=True)
    df['title'].fillna('unknowntitle', inplace=True)

    df['weekday'] = pd.to_datetime(df['activation_date']).dt.weekday
 #   df['dayofmonth'] = pd.to_datetime(df['activation_date']).dt.day
    
    for col in ['description', 'title']:
        df['num_words_' + col] = df[col].apply(lambda comment: len(comment.split()))
        df['num_unique_words_' + col] = df[col].apply(lambda comment: len(set(w for w in comment.split())))

    df['words_vs_unique_title'] = df['num_unique_words_title'] / df['num_words_title'] * 100
    df['words_vs_unique_description'] = df['num_unique_words_description'] / df['num_words_description'] * 100
    
    df['city'] = df['region'] + '_' + df['city']
    df['num_desc_punct'] = df['description'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
    
    for col in agg_cols:
        df[col].fillna(-1, inplace=True)
#---------------------------------------------
count_vectorizer_title = CountVectorizer(stop_words=stopwords.words('russian'), lowercase=True, min_df=2)

title_counts = count_vectorizer_title.fit_transform(train['title'].append(test['title']))

train_title_counts = title_counts[:len(train)]
test_title_counts = title_counts[len(train):]


count_vectorizer_desc = TfidfVectorizer(stop_words=stopwords.words('russian'), 
                                        lowercase=True, ngram_range=(1, 2),
                                        max_features=17000)

desc_counts = count_vectorizer_desc.fit_transform(train['description'].append(test['description']))

train_desc_counts = desc_counts[:len(train)]
test_desc_counts = desc_counts[len(train):]

#---------------------------------------------
target = 'deal_probability'
predictors = [
    'num_desc_punct',
    'words_vs_unique_description', 'num_unique_words_description', 'num_unique_words_title', 'num_words_description', 'num_words_title',
    'avg_times_up_user', 'avg_days_up_user', 'n_user_items', 
    'price', 'item_seq_number', 'user_price_aver', 'user_price_std', 'user_seq_aver', 'user_seq_std',
    'user_id_parent_category_name_category_name_count', 
    'user_id_parent_category_name_category_name_aver_pri',
    'region_city_parent_category_name_category_name_count','rnn_pred','t','d','ridge_pred','cb_pred'
]
categorical = [
    'image_top_1', 'param_1', 'param_2', 'param_3', 
    'city', 'region', 'category_name', 'parent_category_name', 'user_type'
]
predictors = predictors + categorical
#--------------------------------------------
for feature in categorical:
    print(f'Transforming {feature}...')
    encoder = LabelEncoder()
    train[feature].fillna('unknown',inplace=True)
    test[feature].fillna('unknown',inplace=True)
    encoder.fit(train[feature].append(test[feature]).astype(str))
    
    train[feature] = encoder.transform(train[feature].astype(str))
    test[feature] = encoder.transform(test[feature].astype(str))
#--------------------------------------------------------
train["price"] = np.log(train["price"]+0.001)
train["price"].fillna(-999,inplace=True)
train["image_top_1"].fillna(-999,inplace=True)

test["price"] = np.log(test["price"]+0.001)
test["price"].fillna(-999,inplace=True)
test["image_top_1"].fillna(-999,inplace=True)
#--------------------------------------------
feature_names = np.hstack([
    count_vectorizer_desc.get_feature_names(),
    count_vectorizer_title.get_feature_names(),
    predictors
])
print('Number of features:', len(feature_names))

X = scipy.sparse.hstack([
    train_desc_counts,
    train_title_counts,
    train.loc[: , predictors]
], format='csr')
y = train.deal_probability

del test, train_desc_counts, train_title_counts, train, test_desc_counts, test_title_counts
gc.collect()

Transforming image_top_1...
Transforming param_1...
Transforming param_2...
Transforming param_3...
Transforming city...
Transforming region...
Transforming category_name...
Transforming parent_category_name...
Transforming user_type...
Number of features: 122989


42

In [ ]:
def lgb_evaluate(min_child_weight,
                 colsample_bytree,
                 num_leaves,
                 subsample,
                 min_split_gain,
                 reg_alpha):


    cv_result = cross_val_score(LGBMRegressor(min_child_weight= int(min_child_weight),
                                            colsample_bytree=max(min(colsample_bytree, 1), 0),
                                            num_leaves=int(num_leaves),
                                            subsample=max(min(subsample, 1), 0),
                                            min_split_gain=max(min_split_gain, 0),
                                            reg_alpha = max(reg_alpha, 0)
                                            ),
                                X, y, 'neg_mean_squared_error', cv=5).mean()
    return cv_result


num_iter = 20
init_points = 5

lgbBO = BayesianOptimization(lgb_evaluate, {'min_child_weight': (0, 10),
                                                'colsample_bytree': (0.1, 1),
                                                'num_leaves': (8, 1024),
                                                'subsample': (0.5, 1),
                                                'min_split_gain': (0, 10),
                                                'reg_alpha': (0, 10),
                                                })

lgbBO.maximize(init_points=init_points, n_iter=num_iter)

print('Final Results')
print('lgb: %f' % lgbBO.res['max']['max_val'])
print(lgbBO.res['max']['max_params'])

Initialization
-------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   min_child_weight |   min_split_gain |   num_leaves |   reg_alpha |   subsample | 
    1 | 03m36s |   -0.04760 |             0.2313 |             2.7595 |           1.1188 |     821.1594 |      3.5534 |      0.5726 | 
    2 | 02m58s |   -0.04808 |             0.1631 |             7.4462 |           3.4677 |     782.6866 |      2.0087 |      0.6590 | 
    3 | 02m49s |   -0.04742 |             0.8790 |             4.0844 |           1.3921 |      78.8646 |      9.7490 |      0.9809 | 
    4 | 01m59s |   -0.04781 |             0.8456 |             5.3198 |           4.5433 |     513.2922 |      7.7398 |      0.5787 | 
    5 | 01m46s |   -0.04787 |             0.3728 |             1.0041 |           5.0645 |     130.4369 |      6.4890 |      0.5359 | 


/home/g492652607/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


Bayesian Optimization
-------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   min_child_weight |   min_split_gain |   num_leaves |   reg_alpha |   subsample | 
    6 | 02m43s |   -0.04787 |             0.9158 |             0.2099 |           7.2992 |    1023.3958 |      9.6841 |      0.8116 | 


/home/g492652607/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00024073]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/g492652607/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


    7 | 08m38s |   -0.04692 |             0.7393 |             0.4364 |           0.2137 |     700.4648 |      9.2812 |      0.9946 | 


...

/home/g492652607/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-2.79117376e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 55, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/g492652607/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


   13 | 03m18s |   -0.04723 |             0.9898 |             3.6146 |           0.6539 |     200.5611 |      4.3133 |      0.5436 | 


/home/g492652607/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.00152729]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 47, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/g492652607/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


   14 | 15m33s |   -0.04675 |             0.9137 |             0.2652 |           0.0153 |     671.0130 |      3.5482 |      0.7819 | 


/home/g492652607/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00073346]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 47, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/g492652607/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00146461]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/g492652607/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00053295]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/g492652607/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_proc